In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2ap4iSqM3AnMiWXVgMXApXnykxn_39mZ1f4xMB3rYrFc93XaD

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import tensorflow
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm
from PIL import Image
import pandas as pd
import numpy as np
import pickle
import random
import cv2
import csv
import os

In [ ]:
template_folder = '/content/drive/MyDrive/projet_python/codesource/templates'
static_folder = '/content/drive/MyDrive/projet_python/codesource/static'
chemin_fichier_csv = "/content/drive/MyDrive/projet_python/codesource/fashion.csv"
chemin_featurelist_file = "/content/drive/MyDrive/projet_python/codesource/featurevector.pkl"
chemin_fileNames_file = "/content/drive/MyDrive/projet_python/codesource/filenames.pkl"

app = Flask(__name__, template_folder=template_folder, static_folder=static_folder)
run_with_ngrok(app)

app.config['UPLOAD_FOLDER'] = "/content/drive/MyDrive/projet_python/codesource/static/uploads"


In [ ]:
feature_list = np.array(pickle.load(open(chemin_featurelist_file, 'rb')))
filenames = pickle.load(open(chemin_fileNames_file, 'rb'))

model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

In [ ]:
def obtenir_valeurs_distinctes_csv(chemin_fichier):
    dataframe = pd.read_csv(chemin_fichier)
    colonnes = ["Gender", "Category", "Usage"]
    valeurs_distinctes = {}
    for colonne in colonnes:
        valeurs_distinctes[colonne] = dataframe[colonne].unique()
    return valeurs_distinctes

In [ ]:
def select_and_shuffle_columns(input_csv_file):
    selected_columns = ["Gender", "Category", "ProductType", "Usage", "ProductTitle", "Image"]

    with open(input_csv_file, 'r', encoding='utf-8') as input_file:
        reader = csv.DictReader(input_file)
        fieldnames = reader.fieldnames
        selected_products = [{col: row[col] for col in selected_columns} for row in reader]
        random.shuffle(selected_products)
    return selected_products

In [ ]:
def filter_csv(file_path, column, value):
        df = pd.read_csv(file_path)
        filtered_rows = df[df[column] == value]
        filtered_data = filtered_rows.to_dict('records')
        random.shuffle(filtered_data)
        return filtered_data

In [ ]:
def get_last_uploaded_image():
    files = [f for f in os.listdir(app.config['UPLOAD_FOLDER']) if os.path.isfile(os.path.join(app.config['UPLOAD_FOLDER'], f))]
    files.sort(key=lambda x: os.path.getmtime(os.path.join(app.config['UPLOAD_FOLDER'], x)), reverse=True)
    if files:
        return os.path.join(app.config['UPLOAD_FOLDER'], files[0])
    else:
        return None

In [ ]:
def extract_feature(img_path, model):
    img=cv2.imread(img_path)
    img=cv2.resize(img, (224,224))
    img=np.array(img)
    expand_img=np.expand_dims(img, axis=0)
    pre_img=preprocess_input(expand_img)
    result=model.predict(pre_img).flatten()
    normalized=result/norm(result)
    return normalized

In [ ]:
def recommend(features, feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)
    distances, indices = neighbors.kneighbors([features])
    return indices

In [ ]:
valeurs_distinctes = obtenir_valeurs_distinctes_csv(chemin_fichier_csv)
shuffled_selected_products = select_and_shuffle_columns(chemin_fichier_csv)
print(valeurs_distinctes)

{'Gender': array(['Girls', 'Boys', 'Men', 'Women'], dtype=object), 'Category': array(['Topwear', 'Bottomwear', 'Dress', 'Innerwear', 'Socks',
       'Apparel Set', 'Shoes', 'Flip Flops', 'Sandal'], dtype=object), 'Usage': array(['Casual', 'Ethnic', 'Sports', 'Formal', 'Smart Casual', 'Party'],
      dtype=object)}


In [ ]:
@app.route('/')
def home():
    return render_template('home.html', colonnes=valeurs_distinctes, products=shuffled_selected_products, css_file='home')

@app.route('/upload', methods=['POST'])
def upload_img():
    uploaded_file = request.files['file']
    if uploaded_file.filename != '':
        file_path = os.path.join('/content/drive/MyDrive/ColabNotebooks/python_project/static/uploads', uploaded_file.filename)
        uploaded_file.save(file_path)
        display_image = Image.open(uploaded_file)
        resized_img = display_image.resize((200, 200))
        features = extract_feature(file_path, model)
        indices = recommend(features, feature_list)
        uploaded_file = get_last_uploaded_image().replace('/content/drive/MyDrive/ColabNotebooks/python_project/static/', '')
        return render_template('result.html', img_path=file_path, indices=indices, filenames=filenames, colonnes=valeurs_distinctes, css_file='result', uploaded_file=uploaded_file)
    return render_template('home.html', colonnes=valeurs_distinctes, products=shuffled_selected_products, css_file='home')

@app.route('/filtredProducts', methods=['GET'])
def filter_route():
    url_parts = request.url.split('?')
    query_part = url_parts[1] if len(url_parts) > 1 else ""
    query_params = query_part.split('=')
    column = query_params[0] if len(query_params) > 0 else ""
    value = query_params[1] if len(query_params) > 1 else ""
    value = value.replace("+", " ")
    filtred_products = filter_csv(chemin_fichier_csv, column, value)
    return render_template('filtred_products.html', column=column, value=value, products=filtred_products, colonnes=valeurs_distinctes, css_file='home')

In [ ]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll